# Lab 8 : T5

@copyright: 
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. T5 model
2. T5 model for summarization

## 1. T5 model

[T5](https://arxiv.org/abs/1910.10683)는 Text-to-Text Transfer Transformer의 약자로, NLP task를 하나의 text-to-text 문제로 통일하여 학습하는 모델이다.

T5는 11가지의 NLP task를 하나의 모델로 학습시킨다.
- Translation 
- Question Answering 
- Classification 
- Summarization 
- Paraphrasing 
- etc. 

![T5](./figure/t5.png)

T5는 BERT와 같이 Transformer Encoder를 사용하며, BERT와 다르게 Decoder를 사용한다.
즉, Seq2Seq 모델과 같이 Encoder-Decoder 구조를 가진다. 따라서, output으로 sequence를 생성할 수 있다.

본 실습에서는 T5를 이용하여 Seq2Seq task인 Summarization task를 수행해본다.

T5 model의 tokenizing을 위해, [TorchText](https://pytorch.org/text/stable/transforms.html#sentencepiecetokenizer)를 사용한다.

TorchText는 PyTorch에서 제공하는 NLP library로, 다양한 NLP task를 수행할 수 있는 기능을 제공한다.

TorchText의 [SentencePieceTokenizer](https://github.com/google/sentencepiece)를 이용하여, T5 model의 input과 output을 tokenizing한다.

In [ ]:
from torchtext.models import T5Transform

padding_idx = 0
eos_idx = 1
max_seq_len = 512
t5_sp_model_path = "https://download.pytorch.org/models/text/t5_tokenizer_base.model"

transform = T5Transform(
    sp_model_path=t5_sp_model_path,
    max_seq_len=max_seq_len,
    eos_idx=eos_idx,
    padding_idx=padding_idx,
)

# example
print(transform(["Hi this is I know lab NLP course!", "1 represent <EOS> token, and 0 represent <PAD> token"]))


## 2. T5 model for summarization



Dataset은 [CNN/Daily Mail](https://pytorch.org/text/stable/datasets.html.)으로, CNN과 Daily Mail의 기사를 학습 데이터로 사용한다.

In [ ]:
from functools import partial

from torch.utils.data import DataLoader
from torchtext.datasets import CNNDM

cnndm_batch_size = 5
cnndm_datapipe = CNNDM(split="test")
task = "summarize"

def apply_prefix(task, x):
    return f"{task}: " + x[0], x[1]

cnndm_datapipe = cnndm_datapipe.map(partial(apply_prefix, task))
cnndm_datapipe = cnndm_datapipe.batch(cnndm_batch_size)
cnndm_datapipe = cnndm_datapipe.rows2columnar(["article", "abstract"])
cnndm_dataloader = DataLoader(cnndm_datapipe, shuffle=True, batch_size=None)

# example
next(iter(cnndm_dataloader))

In [ ]:
from torchtext.models import T5_BASE_GENERATION
from torchtext.prototype.generate import GenerationUtils

t5_base = T5_BASE_GENERATION
transform = t5_base.transform()
model = t5_base.get_model()
model.eval()

sequence_generator = GenerationUtils(model)

In [ ]:
batch = next(iter(cnndm_dataloader))
input_text = batch["article"]
target = batch["abstract"]
beam_size = 1

model_input = transform(input_text)
model_output = sequence_generator.generate(model_input, eos_idx=eos_idx, num_beams=beam_size)
output_text = transform.decode(model_output.tolist())

for i in range(cnndm_batch_size):
    print(f"Example {i+1}:\n")
    print(f"prediction: {output_text[i]}\n")
    print(f"target: {target[i]}\n\n")